In [25]:
import json
import pandas as pd
import glob


In [48]:
tasks = {}

tasks['Examples'] = []
tasks['Categories'] = []
for task_path in glob.glob("/home/gujiashe/Tk-Instruct/data/tasks/*.json"):
    with open(task_path, "r") as task_f:
        s = task_f.read()
        task = json.loads(s)
        pos_samples = "Positive Examples: " + '\n\n'
        for example in task['Positive Examples']:
            sample = ""
            for k in example:
                sample += k + ': ' + '\n\n'
                sample += example[k] + '\n\n'
            sample += '\n\n'
            pos_samples += sample

        neg_samples = "Negative Examples: " + '\n\n'
        for example in task['Negative Examples']:
            sample = ""
            for k in example:
                sample += k + ': ' + '\n\n'
                sample += example[k] + '\n\n'
            sample += '\n\n'
            neg_samples += sample
        samples = pos_samples + neg_samples
        Categories = task['Categories'][0]
    tasks['Examples'].append(samples)
    tasks['Categories'].append(Categories)
tasks = pd.DataFrame(tasks)

In [60]:
tasks

,Examples,Categories
0,Positive Examples: \n\ninput: \n\nczy w pobliż...,Translation
1,Positive Examples: \n\ninput: \n\nTomber amour...,Sentence Perturbation
2,Positive Examples: \n\ninput: \n\nI have bough...,Text Matching
3,Positive Examples: \n\ninput: \n\nRoman Yakub ...,Question Answering
4,Positive Examples: \n\ninput: \n\nBob helped h...,Linguistic Probing
...,...,...
1608,Positive Examples: \n\ninput: \n\nText: Abbott...,Text Categorization
1609,"Positive Examples: \n\ninput: \n\n1, 3, ['238'...",Program Execution
1610,Positive Examples: \n\ninput: \n\nItaly have d...,Translation
1611,Positive Examples: \n\ninput: \n\nI work 40 ho...,Text Categorization


In [57]:
tasks.to_csv("tasks.csv", sep = '\t', index=None)

In [59]:
pd.read_csv("tasks.csv", sep = '\t')

,Examples,Categories
0,Positive Examples: \n\ninput: \n\nczy w pobliż...,Translation
1,Positive Examples: \n\ninput: \n\nTomber amour...,Sentence Perturbation
2,Positive Examples: \n\ninput: \n\nI have bough...,Text Matching
3,Positive Examples: \n\ninput: \n\nRoman Yakub ...,Question Answering
4,Positive Examples: \n\ninput: \n\nBob helped h...,Linguistic Probing
...,...,...
1608,Positive Examples: \n\ninput: \n\nText: Abbott...,Text Categorization
1609,"Positive Examples: \n\ninput: \n\n1, 3, ['238'...",Program Execution
1610,Positive Examples: \n\ninput: \n\nItaly have d...,Translation
1611,Positive Examples: \n\ninput: \n\nI work 40 ho...,Text Categorization
